In [ ]:
import tkinter as tk
from PIL import Image, ImageTk
import glob
import os
import numpy as np
import json
import cv2

from tkinter.filedialog import askdirectory, askopenfilename

def selectPath():
    annPathVar.set(askopenfilename())
    #imgPathVar.set('/home/ubuntu/Data/autoFrameFilter/test/images/test_29000_50.jpg')
    show_img_pose_label()
    #print(imgPathVar.get(),humanCountVar.get(),humanNumVar.get())


def draw_one_alphapose_label(frame, oneResult, format='coco'):
    if format == 'coco':
        l_pair = [(0, 1), (0, 2), (1, 3), (2, 4),  # Head
            (5, 6), (5, 7), (7, 9), (6, 8), (8, 10),(17, 11), (17, 12),  # Body
            (11, 13), (12, 14), (13, 15), (14, 16)]
        p_color = [(0, 255, 255),(0, 191, 255),(0, 255, 102),(0, 77, 255), (0, 255, 0), 
                   #Nose, LEye, REye, LEar, REar
                  (77,255,255),(77, 255, 204), (77,204,255), (191, 255, 77), (77,191,255), (191, 255, 77), 
                   #LShoulder, RShoulder, LElbow, RElbow, LWrist, RWrist
                  (204,77,255),(77,255,204), (191,77,255), (77,255,191), (127,77,255), (77,255,127), (0, 255, 255)] 
                   #LHip, RHip, LKnee, Rknee, LAnkle, RAnkle, Neck
        line_color = [(0, 215, 255), (0, 255, 204), (0, 134, 255), (0, 255, 50), 
                    (77,255,222), (77,196,255), (77,135,255), (191,255,77), (77,255,77), 
                    (77,222,255), (255,156,127), 
                    (0,127,255), (255,127,77), (0,77,255), (255,77,36)]
    img = frame
    part_line = {}
    kp_preds = np.array(oneResult['keypoints'])
    kp_scores = np.array(oneResult['kp_score'])
    #labelNameVar.set(oneResult.get('categroie',' ')+' : '+oneResult.get('categroieName','empty'))
    # Draw keypoints
    for n in range(len(kp_scores)):
        if kp_scores[n] <= 0.05:
            continue
        cor_x, cor_y = int(kp_preds[n, 0]), int(kp_preds[n, 1])
        part_line[n] = (cor_x, cor_y)
        cv2.circle(img, (cor_x, cor_y), 4, p_color[n], -1)
    # Draw limbs
    for i, (start_p, end_p) in enumerate(l_pair):
        if start_p in part_line and end_p in part_line:
            start_xy = part_line[start_p]
            end_xy = part_line[end_p]
            cv2.line(img, start_xy, end_xy, line_color[i], 2*(kp_scores[start_p] + kp_scores[end_p]) + 1)
    return img

def cut_one_alphapose(frame, oneResult):
    img=frame
    kp_preds = np.array(oneResult['keypoints'])
    kp_scores = np.array(oneResult['kp_score'])
    h,w,_=img.shape
    #print(img.shape)
    minX, minY =w,h
    maxX, maxY =0,0
    for n in range(len(kp_scores)):
        if kp_scores[n] <= 0.05:
            continue
        minX = min(kp_preds[n,0], minX)
        minY = min(kp_preds[n,1], minY)
        maxX = max(kp_preds[n,0], maxX)
        maxY = max(kp_preds[n,1], maxY)
    assert maxX>minX, print('error : maxX<=minX')
    assert maxY>minY, print('error : maxY<=minY')
    cutW=maxX-minX
    cutH=maxY-minY
    minX = int(max(0,minX-0.3*cutW))
    maxX = int(min(w,maxX+0.3*cutW))
    minY = int(max(0,minY-0.3*cutH))
    maxY = int(min(h,maxY+0.3*cutH))
    imgCut = img[minY:maxY, minX:maxX]
    cutH, cutW, _ = imgCut.shape
    maxCutW, maxCutH = 500, 300
    if cutH/cutW >maxCutH/maxCutW:
        reH = maxCutH
        reW = int(maxCutH*cutW/cutH)
    else:
        reW = maxCutW
        reH = int(maxCutW*cutH/cutW)
    imgCut = cv2.resize(imgCut,(reW,reH))
    
    return imgCut


def callback(cateNum, cateName):
    with open(annPathVar.get(),'r',encoding='utf-8') as jsonfile:
        annResult = json.load(jsonfile)
    annResult[humanCountVar.get()]['categorie']=cateNum
    annResult[humanCountVar.get()]['categorieName']=cateName
    with open(annPathVar.get(),'w',encoding='utf-8') as jsonfile:
        json.dump(annResult,jsonfile)
    if humanCountVar.get()+1<humanNumVar.get():
        humanCountVar.set(humanCountVar.get()+1)
    else:
        humanCountVar.set(0)
    show_img_pose_label()
    
    

def callbackNext(addNum):
    humanCount = humanCountVar.get()+addNum
    if humanCount+1>=humanNumVar.get():
        humanCountVar.set(0)
    elif humanCount+1 <=0:
        humanCountVar.set(humanNumVar.get()-1)
    else:
        humanCountVar.set(humanCount)
    show_img_pose_label()
    
        
def show_img_pose_label():
    annPath = annPathVar.get()
    annDir, annName = os.path.split(annPathVar.get())
    dataDir =  os.path.split(annDir)[0]
    kpDir = os.path.join(dataDir, 'keypoints')
    imgDir = os.path.join(dataDir, 'images')
    imgName= annName[:-5]+'.jpg'
    imgPath = os.path.join(imgDir,imgName)
    imgPathVar.set(imgPath)
    assert os.path.exists(imgPath), print('error : img not exits')
    with open(annPath,'r',encoding='utf-8') as jsonfile:
        annResult = json.load(jsonfile)
    
    humanKp=annResult[humanCountVar.get()]
    img = cv2.imread(imgPathVar.get())
    imgCut = cut_one_alphapose(img, humanKp)
    cv2.imwrite('./temp/tempCut.jpg', imgCut)
    imgCut = Image.open('./temp/tempCut.jpg')  
    imgCut = ImageTk.PhotoImage(imgCut)
         
    imgKp = draw_one_alphapose_label(img, humanKp)
    cv2.imwrite('./temp/temp.jpg', imgKp)
    imgKp = Image.open('./temp/temp.jpg')  
    imgKp = imgKp.resize((711,400))
    imgKp = ImageTk.PhotoImage(imgKp)
    
    imgLabel.configure(image=imgKp)
    imgCutLabel.configure(image=imgCut)
    humanNumVar.set(len(annResult))
    #print(annResult[humanCountVar.get()])
    try:
        label = str(annResult[humanCountVar.get()]['categorie'])+' : '+str(annResult[humanCountVar.get()]['categorieName'])
    except:
        label = 'no label'
    labelNameVar.set(label)
    imgNameLabel.configure(text='%s     human: %d/%d'%(annPathVar.get(), humanCountVar.get()+1, humanNumVar.get()))
    root.mainloop()
    
    
root = tk.Tk()
root.title('show pose label')

#define vars
annPathVar = tk.StringVar()
imgPathVar = tk.StringVar()
humanNumVar = tk.IntVar()
humanCountVar = tk.IntVar()
labelNameVar = tk.StringVar()
annPathVar = tk.StringVar()

#init vars
annPathVar.set(None)
imgPathVar.set(None)
photo = None
photoCut = None
humanCountVar.set(0)
humanNumVar.set(0)
labelNameVar.set('label')
annPathVar.set(None)

#show img and pose label frame 
frameImg = tk.Frame(root)
imgLabel = tk.Label(frameImg, image=photo, )
imgLabel.pack()
imgCutLabel = tk.Label(frameImg, image=photoCut, height=300)
imgCutLabel.pack(side=tk.BOTTOM)
poseLabel = tk.Label(frameImg, textvariable=labelNameVar)
poseLabel.pack(side=tk.RIGHT)

#info frame
frameInfo = tk.Frame(root)
imgNameLabel  = tk.Label(frameInfo, text='%s     human: %d/%d'%(annPathVar.get(), humanCountVar.get(), humanNumVar.get()))
imgNameLabel.pack()

#path entry frame
framePath = tk.Frame(root)
pathEntry = tk.Entry(framePath, textvariable = annPathVar, width=40)
pathEntry.pack(side=tk.LEFT)
pathButton = tk.Button(framePath, text = "add annotation filepath", command = selectPath)
pathButton.pack(side=tk.RIGHT)

#button frame
frameButton = tk.Frame(root)
theButton1 = tk.Button(frameButton, text="0: stand up   ", height=4, activebackground = 'GREEN', command=lambda:callback(0,'stand up'))
theButton1.pack()
theButton2 = tk.Button(frameButton, text="1: read&write ", height=4, activebackground = 'GREEN', command=lambda:callback(1,'read&write'))
theButton2.pack()
theButton3 = tk.Button(frameButton, text="2: listening  ", height=4, activebackground = 'GREEN', command=lambda:callback(2,'listening'))
theButton3.pack()
theButton4 = tk.Button(frameButton, text="3: hand up    ", height=4, activebackground = 'GREEN', command=lambda:callback(3,'hand up'))
theButton4.pack()
theButton5 = tk.Button(frameButton, text="4: turn around", height=4, activebackground = 'GREEN', command=lambda:callback(4,'turn around'))
theButton5.pack()
theButton6 = tk.Button(frameButton, text="5: sleep      ", height=4, activebackground = 'GREEN', command=lambda:callback(5,'sleep'))
theButton6.pack()
theButton7 = tk.Button(frameButton, text="6: else       ", height=4, activebackground = 'GREEN', command=lambda:callback(6,'else'))
theButton7.pack()

nextButton = tk.Button(frameButton, text="next", height=4, activebackground = 'GREEN', command=lambda:callbackNext(1))
nextButton.pack(side=tk.RIGHT)
prevButton = tk.Button(frameButton, text="prev", height=4, activebackground = 'GREEN', command=lambda:callbackNext(-1))
prevButton.pack(side=tk.LEFT)



framePath.pack(side=tk.TOP)
frameInfo.pack(side=tk.TOP)
frameImg.pack(side=tk.LEFT)
frameButton.pack(side=tk.RIGHT)




#start
root.mainloop()